In [ ]:
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.io 
import random
from networks.DMGP import*
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Will use only the first and the second GPU devices

np.random.seed(0)
tf.set_random_seed(0)

In [ ]:
f = open('./JURA/jura.txt', 'r')
D = []
for line in f:
    D.append(line.split())
D = np.asarray(D).astype(np.float)
f.close()

Xtrain_raw = D[:,0:2]
Ytrain_raw = D[:,4:]

Xmean = np.mean(Xtrain_raw, axis=0, keepdims=True)
Xstd = np.std(Xtrain_raw, axis=0, keepdims=True)
Xtrain = (Xtrain_raw - Xmean) / Xstd

Ymean = np.mean(Ytrain_raw, axis=0, keepdims=True)
Ystd = np.max(np.std(Ytrain_raw, axis=0, keepdims=True))
Ytrain = (Ytrain_raw - Ymean) / Ystd

#====================================================================

f = open('./JURA/jura_test.txt', 'r')
D = []
for line in f:
    D.append(line.split())
D = np.asarray(D).astype(np.float)
f.close()

Xtest_raw = D[:,0:2]
Ytest_raw = D[:,4:]

Xtest = (Xtest_raw - Xmean) / Xstd
Ytest = (Ytest_raw - Ymean) / Ystd
Dout = 7

In [ ]:
# Define DMGP parameters
N_LAYER = 2 # the number of layers
D = [2, Dout, Dout] # dimensions for each layer
M = [20, 20] # the number of pseudo-inputs for each layer

K = 20 # the number of samples

DGP_net = DGP_DVSI(N_LAYER, D, M, K, Mat=True, MCO=True)

In [ ]:
elbo_hist, elbo_hist_test = DGP_net.train(Xtrain,Ytrain,Xtest,Ytest,epoch=1000)

In [ ]:
Yest_mean, Yest_var = DGP_net.predict(Xtest, K=1000)